In [1]:
import numpy as np

f = "/media/george-vengrovski/disk2/training_song_detector/wav_and_npz_files/USA5177_45268.29527473_12_8_8_12_7.npz"

f = np.load(f, allow_pickle=True)

print(f.files)
# hdbscan_labels, dataloader_indices, sample_ids, data_dir_mapping

print(f['s'].shape)
print(f['song'].shape)
print(f['labels'].shape)

['s', 'labels', 'song']


KeyError: 'hdbscan_labels is not a file in the archive'